In [51]:
import sagemaker
import os
import boto3
import sagemaker
import pandas as pd
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [52]:
bucket = sagemaker.Session().default_bucket()

In [53]:
bucket

'sagemaker-us-east-1-087946768277'

In [75]:
url="https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/d546eaee765268bf2f487608c537c05e22e4b221/iris.csv"
data = pd.read_csv(url)
data.columns = ['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width','Species']

In [76]:
data.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [79]:
# Train/test split, 70%-30%
train_data = data.sample(frac=0.7, random_state=42)
test_data = data.drop(train_data.index)
test_data.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
6,4.6,3.4,1.4,0.3,setosa
14,5.8,4.0,1.2,0.2,setosa
20,5.4,3.4,1.7,0.2,setosa


In [78]:
# Write to csv
train_data.to_csv('iris_train.csv', index=False)
test_data.to_csv('iris_test.csv', index=False)

In [93]:
train_data.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
73,6.1,2.8,4.7,1.2,versicolor
18,5.7,3.8,1.7,0.3,setosa
118,7.7,2.6,6.9,2.3,virginica
78,6.0,2.9,4.5,1.5,versicolor
76,6.8,2.8,4.8,1.4,versicolor


In [11]:
!ls

Untitled.ipynb	 Untitled5.ipynb		   rdeploymentpipeline
Untitled1.ipynb  Untitled6.ipynb		   rdeploymentpipeline.ipynb
Untitled2.ipynb  epel-release-latest-7.noarch.rpm  seeker-fraud-model
Untitled3.ipynb  iris_test.csv			   untitled.md
Untitled4.ipynb  iris_train.csv			   yavapai-poc


In [80]:
bucket = 'testrdeployment'
prefix = 'sagemaker/DEMO-hpo-r-byo'

In [81]:
train_file = 'iris_train.csv'
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', train_file)).upload_file(train_file)

In [99]:
algorithm_name = 'testrdeployment'

In [83]:
region = boto3.Session().region_name
region

'us-east-1'

In [84]:
account = boto3.client('sts').get_caller_identity().get('Account')
account

'087946768277'

In [85]:
role = sagemaker.get_execution_role()
role

Couldn't call 'get_role' to get Role ARN from role name yavapai-sagemaker-fullaccess to get Role path.


'arn:aws:iam::087946768277:role/yavapai-sagemaker-fullaccess'

In [86]:
role

'arn:aws:iam::087946768277:role/yavapai-sagemaker-fullaccess'

In [87]:
estimator = sagemaker.estimator.Estimator(
    image_uri='{}.dkr.ecr.{}.amazonaws.com/testrdeployment:1'.format(account, region),
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker.Session()) 

In [89]:
test = estimator.fit({'train': 's3://{}/{}/train'.format(bucket, prefix)})

2021-02-04 00:13:06 Starting - Starting the training job...
2021-02-04 00:13:29 Starting - Launching requested ML instancesProfilerReport-1612397585: InProgress
.........
2021-02-04 00:14:50 Starting - Preparing the instances for training...
2021-02-04 00:15:31 Downloading - Downloading input data...
2021-02-04 00:15:51 Training - Downloading the training image...
2021-02-04 00:16:32 Uploading - Uploading generated training model
2021-02-04 00:16:32 Completed - Training job completed
Loading required package: class
Loaded mda 0.4-10

Loading required package: methods
Warning message:
In is.na(x) : is.na() applied to non-(list or vector) of type 'NULL'
               Length Class  Mode   
call             6    -none- call   
all.terms       15    -none- numeric
selected.terms   7    -none- numeric
penalty          1    -none- numeric
degree           1    -none- numeric
nk               1    -none- numeric
thresh           1    -none- numeric
gcv              1    -none- numeric
factor 

In [94]:
training_job_name = estimator.latest_training_job.name

In [95]:
print(training_job_name)

testrdeployment-2021-02-04-00-13-05-918


In [103]:
best_model_s3 = boto3.client('sagemaker').describe_training_job(TrainingJobName = training_job_name)['ModelArtifacts']['S3ModelArtifacts']

In [104]:
best_model_s3

's3://testrdeployment/sagemaker/DEMO-hpo-r-byo/output/testrdeployment-2021-02-04-00-13-05-918/output/model.tar.gz'

In [106]:
import time
r_job = 'DEMO-r-byo-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
r_job

'DEMO-r-byo-2021-02-04-03-26-30'

In [108]:
r_hosting_container = {
    'Image': '{}.dkr.ecr.{}.amazonaws.com/{}:1'.format(account, region, algorithm_name),
    'ModelDataUrl': best_model_s3
}

In [109]:
create_model_response = boto3.client('sagemaker').create_model(
    ModelName=r_job,
    ExecutionRoleArn=role,
    PrimaryContainer=r_hosting_container)

In [110]:
print(create_model_response['ModelArn'])

arn:aws:sagemaker:us-east-1:087946768277:model/demo-r-byo-2021-02-04-03-26-30


In [111]:
r_endpoint_config = 'DEMO-r-byo-config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(r_endpoint_config)

DEMO-r-byo-config-2021-02-04-03-27-44


In [112]:
create_endpoint_config_response = boto3.client('sagemaker').create_endpoint_config(
    EndpointConfigName=r_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.t2.medium',
        'InitialInstanceCount': 1,
        'ModelName': r_job,
        'VariantName': 'AllTraffic'}])
print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint Config Arn: arn:aws:sagemaker:us-east-1:087946768277:endpoint-config/demo-r-byo-config-2021-02-04-03-27-44


In [115]:
r_endpoint = 'DEMO-r-endpoint-' + time.strftime("%Y%m%d%H%M", time.gmtime())
print(r_endpoint)
create_endpoint_response = boto3.client('sagemaker').create_endpoint(
    EndpointName=r_endpoint,
    EndpointConfigName=r_endpoint_config)
print(create_endpoint_response['EndpointArn'])

DEMO-r-endpoint-202102040329
arn:aws:sagemaker:us-east-1:087946768277:endpoint/demo-r-endpoint-202102040329
CPU times: user 20.4 ms, sys: 0 ns, total: 20.4 ms
Wall time: 237 ms


In [116]:
resp = boto3.client('sagemaker').describe_endpoint(EndpointName=r_endpoint)
status = resp['EndpointStatus']
print("Status: " + status)

Status: Creating


In [117]:
%%time

try:
    boto3.client('sagemaker').get_waiter('endpoint_in_service').wait(EndpointName=r_endpoint)
finally:
    resp = boto3.client('sagemaker').describe_endpoint(EndpointName=r_endpoint)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Status: " + status)

    if status != 'InService':
        raise Exception('Endpoint creation did not succeed')

Arn: arn:aws:sagemaker:us-east-1:087946768277:endpoint/demo-r-endpoint-202102040329
Status: InService


In [118]:
import pandas as pd
import json

iris_test = pd.read_csv('iris_test.csv')

runtime = boto3.Session().client('runtime.sagemaker')

In [119]:
%%time 

# there is a limit of max 500 samples at a time for invoking endpoints
payload = iris_test.drop(['Sepal.Length'], axis=1).to_csv(index=False)

response = runtime.invoke_endpoint(EndpointName=r_endpoint,
                                   ContentType='text/csv',
                                   Body=payload)

result = json.loads(response['Body'].read().decode())
display(result)

['4.6534264908056,4.78049539134011,4.95570696938643,5.26177767757299,5.08744538495788,5.19656793386068,4.80267161529316,3.8521241516219,5.30613012547908,5.19656793386068,6.27353076781652,6.29570699176956,6.08873120527164,5.18932821808307,6.1138103289138,5.68604089909805,5.95320658233716,6.16273895883271,5.78011195157285,5.96702443915705,5.41759692086013,5.83538337885242,5.90678586478246,5.92566822396419,6.13888945255596,5.69985875591794,5.83154849385598,7.07781819027535,6.89768251409641,6.55781428342878,5.95453355496938,7.32169625785666,6.46611122119992,6.43091165903652,6.53403473821116,6.03927028108731,6.71121329761144,6.12368077282666,6.79567191577384,7.04497545709314,6.51527101772198,6.60309662810294,6.61563749986521,6.34949024087291,6.34835186799237']

CPU times: user 13.6 ms, sys: 3.53 ms, total: 17.1 ms
Wall time: 159 ms


In [120]:
boto3.client('sagemaker').delete_endpoint(EndpointName=r_endpoint)

{'ResponseMetadata': {'RequestId': 'f6dd0fc2-132e-4255-b304-ded4360bd429',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f6dd0fc2-132e-4255-b304-ded4360bd429',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 04 Feb 2021 03:41:22 GMT'},
  'RetryAttempts': 0}}